# Dynamic Programming

DP is a way to solve problem with *memory*, like a dp table. One can refer to the previous memory for a sub-solution, instead of calculating again.

A way to think of DP problem is imagine a little bird just tell you the perfect answer `s[n]`, and then you organize how this `s[n]` relates to `s[n-1]` and so on. By so, we can organize the **transformation function**, which is the most important part in DP problems.

For example, the [`Climbing Stairs`](https://leetcode.com/problems/climbing-stairs/description/) problem can be expressed as following:

    s[n] = s[n-1] + s[n-2]
    
where `s[n]` means the number of ways to climbing `n` stairs. This is the definition of the **state** or the **dp table**.

In this case the final **answer** is just the last element of dp table. That is, we only need to follow the rule above. However, in some cases, the last element should be treated differently.

So where did this start? Since everything counts on the previous solution, then we need a **initialization**. At the very begining, it's always easy to think for only 1 or 2 stairs. Generally speaking, one needs to initailize edge cases (first several elements of an array, first several cols/rows of a matrix, so on), sometimes it requires a whole initailization for the whole table to make things easier.

So the four elements to construct an solution to a DP problem:
1. DP table/state definition: how to reduce the problem to smaller ones
2. Transformation function: how to construct the answer with previous solutions
3. Initialization
4. Final answer

## 1-D DP

In [ ]:
class Solution(object):
    def climbStairs(self, n):
        """
        :type n: int
        :rtype: int
        """

**[`Triangle`](https://leetcode.com/problems/triangle/description/)** is a interesting problem showing a triangle like:

    2                 level 1
    3 4
    5 6 7
    8 9 10 11
    ...               level n
    
One needs to find a path from top to bottom with a minimum sum.

- **DFS** -- Reduce to **binary tree**: Triangle structure can be reduced to a binary tree. However, for `n` levels, a tree has `2^n` node while a triangle only has `2n` nodes. That is, we need a `2^n` time to solve this problem, which is inefficient. Note that this inefficiency comes from *calculating the same node for multiple times*.
- **DP** -- Reduce to subproblems of `n-1` levels plus one: `s[n] = min(s[n-1]) + 1`

The **subproblem** concept may be ambifguous, because for DFS the original problem also depends on smaller ones. Note that DFS usually deals with problems whose recursions does *not* have intersections, such as tree problems. On the other hand, the subproblem of DP should be kept for future use, that is *DP table* in practice.

In [1]:
class Solution(object):
    def minimumTotal(self, triangle):
        """
        :type triangle: List[List[int]]
        :rtype: int
        """
        if not triangle: return 0
        
        n = len(triangle)
        # dp definition: the minimum sum at that point
        dp = [[0 for i in range(n)] for j in range(n)]
        # Initialization
        dp[0] = triangle[0]
        for i in range(1,n):
            dp[i][0] = dp[i-1][0] + triangle[i][0]
            dp[i][i] = dp[i-1][i-1] + triangle[i][i]
        
        for i in range(2, n):
            for j in range(1, i): # only the inner elements
                dp[i][j] = min(dp[i-1][j-1], dp[i-1][j])+triangle[i][j]
                
        return min(dp[-1])

[**`Is Subsequence`**](https://leetcode.com/problems/is-subsequence/description/)

DP is natually good at **subsequence** problem. Note that subsequence is a subset with *order*, and elements are not necessarily need to be adjacent. On the other hand, for **subset** problem, there is also chance that DP comes in handy when processing a *sorted* list with order, while the result only needs to be a set.

In [60]:
class Solution(object):
    def isSubsequence(self, s, t):
        """
        :type s: str
        :type t: str
        :rtype: bool
        """
        if not s: return True
        if not t or len(s)>len(t): return False
        
        # dp[i][j] for t[:i+1] contains s[:j+1] if hold
        dp = [[False for j in range(len(s))] for i in range(len(t))]
        # initialzation
        dp[0][0] = s[0]==t[0]
        for i in range(1,len(t)):
            dp[i][0] = (t[i]==s[0] or dp[i-1][0])
        
        # transform
        firstTrue = 0
        for j in range(1,len(s)):
            TrueFound = False
            for i in range(firstTrue+1,len(t)):
                result = dp[i][j-1] and (t[i]==s[j] or dp[i-1][j])
                dp[i][j] = result
                if not TrueFound and result:
                    firstTrue = i
                    TrueFound = True
        
#         for temp in dp:
#             print temp
        
        return dp[-1][-1]
                
                

[**`Longest Increasing Subsequence`**](https://leetcode.com/problems/longest-increasing-subsequence/description/) shows a sequence like:

    1 5 4 2 3 0 --> [1 2 3] --> len 3
    
DP is good at finding **maximun/minimum**, **total** number and answer with a **direction** such as subsequence. *Subsequence* means it can start anywhere, end anywhere and jump to any where but definitely *from left to the right side* with constraint *increasing*. **DP is naturally good at subsequence problem**. The whole problem can be expressed as:

    s[n] = max(s[i] for i in range(0,n) if num[i] < num[n]) + 1

- Definition of `s[n]` the longest increasing subsequence ending up at this node (including node n)
- Think what the initialization value should be here.
- Final answer is *not* the last value of dp table. Why? Not necessarily contain the last number.

In [12]:
class Solution(object):
    def lengthOfLIS(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        if len(nums) < 1:
            return len(nums)
        
        dp = [1]
        for i in range(1, len(nums)):
            prev = [dp[k] for k in range(0,i) if nums[k]<nums[i]]
            if prev:
                dp.append(max(prev) + 1)
            else:
                dp.append(1)
                    
        return max(dp)

[**`Number of Longest Increasing Subsequence`**](https://leetcode.com/problems/number-of-longest-increasing-subsequence/description/) is similar but need a backtracking. First when setting up the dp table, we also keep track of the number that has the same maximum.

For each element (`s[n]`), it's possible there are *multiple valid `max`* value, then all of them should be considered.

In [24]:
class Solution(object):
    def findNumberOfLIS(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        if len(nums) < 1:
            return len(nums)
        
        dp_len = [1 for _ in range(len(nums))]
        dp_num = [1 for _ in range(len(nums))]
        
        for i in range(1, len(nums)): # for the rest
            prev = [(dp_len[k], k) for k in range(0,i) if nums[k]<nums[i]]
            if prev:
                dp_len[i] = max([item[0] for item in prev]) + 1
                dp_num[i] = sum([dp_num[item[1]] for item in prev if item[0]==dp_len[i]-1])
        return sum([dp_num[i] for i in range(len(nums)) if dp_len[i]==max(dp_len)])

[**`Paint House`**](https://leetcode.com/problems/paint-house/description/) gives a row of `n` houses to paint in 3 colors and costs of houses stored in a `n * 3` matrix.

    houses [1 2 3 4 5 ... n]
    r      [               ]
    g      [               ]
    b      [               ]

In [1]:
class Solution(object):
    def minCost(self, costs):
        """
        :type costs: List[List[int]]
        :rtype: int
        """
        n = len(costs)
        if n == 0:
            return 0
        dp = [[costs[0][0]],
              [costs[0][1]],
              [costs[0][2]]]
        
        for i in range(1, n):
            last0, last1, last2 = dp[0][-1], dp[1][-1], dp[2][-1]
            dp[0].append(min(last1, last2) + costs[i][0])
            dp[1].append(min(last0, last2) + costs[i][1])
            dp[2].append(min(last0, last1) + costs[i][2])
            
        return min([dp[0][-1], dp[1][-1], dp[2][-1]])

[**`Paint House II`**](https://leetcode.com/problems/paint-house-ii/description/) gives `k` colors instead of 3. `O(nk)` time to solve.

In [ ]:
class Solution(object):
    def minCostII(self, costs):
        """
        :type costs: List[List[int]]
        :rtype: int
        """
        if not costs: return 0
        n = len(costs)
        k = len(costs[0])
        
        # dp and inital
        dp_prev = [costs[0][i] for i in range(k)]
        dp_curr = []
        for i in range(1, n):
            for j in range(k):
                dp_curr.append(min(dp_prev[:j] + dp_prev[j+1:]) + costs[i][j])
            dp_prev = dp_curr
            dp_curr = []
            
        return min(dp_prev)

The previous solution consumes `O(k)` space. Can we do better?

In [ ]:
class Solution(object):
    def minCostII(self, costs):
        """
        :type costs: List[List[int]]
        :rtype: int
        """

## 2-D DP



1. [`Maximal Square`](https://leetcode.com/problems/maximal-square/description/)
- [`Minimum Path Sum`](https://leetcode.com/problems/minimum-path-sum/description/)
- [`Edit Distance`](https://leetcode.com/problems/edit-distance/description/)
- [`Longest Increasing Path in a Matrix`](https://leetcode.com/problems/longest-increasing-path-in-a-matrix/description/)

`1.` [**`Maximal Square`**](https://leetcode.com/problems/maximal-square/description/) requires one find the maximum square with all-cell 1.

    1 0 1 0 0
    1 0 1 1 1
    1 1 1 1 1
    1 0 0 1 0

In [ ]:
class Solution(object):
    def maximalSquare(self, matrix):
        """
        :type matrix: List[List[str]]
        :rtype: int
        """

`2.` [**`Minimum Path Sum`**](https://leetcode.com/problems/minimum-path-sum/description/) With a `m*n` grid with cost for each cell. You can more down or right at each cell. Find the minimum sum path.

In [ ]:
class Solution(object):
    def minPathSum(self, grid):
        """
        :type grid: List[List[int]]
        :rtype: int
        """

`3.` [**`Edit Distance`**](https://leetcode.com/problems/edit-distance/description/) keeps three types of edit distance as 1:
1. Insert a character
1. Delete a character
1. Replace a character

In [ ]:
class Solution(object):
    def minDistance(self, word1, word2):
        """
        :type word1: str
        :type word2: str
        :rtype: int
        """

`4.` [**`Longest Increasing Path in a Matrix`**](https://leetcode.com/problems/longest-increasing-path-in-a-matrix/description/): **Memorization search**, using `O(n)` memory to tradeoff time to `O(n)`.

- the start of the path can be anywhere.
- not following an order (left to right or vise versa)

At location `i,j`, keep track of `dfs[i,j]` as the start of the sequence

    1  2  3  4  5
    16 17 24 23 6
    15 18 25 22 7
    14 19 20 21 8
    13 12 11 10 1
    
*Q: When should we use memorization search? *
1. transfer function has a lot of direction to explore, not following an order.
2. hard to initialize, not starting from left/right-top/bottom corner

In [ ]:
class Solution(object):
    def longestIncreasingPath(self, matrix):
        """
        :type matrix: List[List[int]]
        :rtype: int
        """